In [ ]:
import os


os.environ['GH_GQL_API_TOKEN'] = 'YOUR_TOKEN_HERE'
print('Setting environment variable: %s' % os.getenv('GH_GQL_API_TOKEN'))

In [2]:
from datetime import datetime

import numpy as np
import pandas as pd
from gql import gql, Client, AIOHTTPTransport
import asyncio
import nest_asyncio
from IPython.display import HTML


# Let nested event loops in Notebook
nest_asyncio.apply()

TOP = 10
users = []

async def main():
    
    token = 'bearer ' + os.getenv('GH_GQL_API_TOKEN')

    # Select your transport with a GitHub url endpoint
    transport = AIOHTTPTransport(
        url="https://api.github.com/graphql",
        headers={'Authorization': token}
    )

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
        ) as session:

        # Provide a GraphQL query
        query = gql(
            """
            query search($query: String!, $type: SearchType!, $numOfResults: Int!, $nextPageCursor: String) {
              search(type: $type, query: $query, first: $numOfResults, after: $nextPageCursor) {
                pageInfo {
                  hasNextPage
                  endCursor
                }
                userCount
                nodes {
                  ... on User {
                    name
                    login
                    location
                    bio
                    url
                    followers {
                      totalCount
                    }
                    following {
                      totalCount
                    }
                    contributionsCollection {
                      totalCommitContributions
                      totalPullRequestContributions
                      totalPullRequestReviewContributions
                    }
                  }
                }
              }
            }
        """)
        
        params = {
          "query": "location:Cuba",
          "type": "USER",
          "numOfResults": 20
        }

        result = await session.execute(query, variable_values=params)
        
        result_users = (result.get('search').get('nodes', [])
                        if result.get('search') else [])
        result_next = (result.get('search').get('pageInfo').get('hasNextPage', False)
                        if result.get('search') else False)
        
        while result_next:
            params["nextPageCursor"] = (
                result.get('search').get('pageInfo').get('endCursor')
            )
            
            await asyncio.sleep(1)
            result = await session.execute(query, variable_values=params)
            result_users += result.get('search').get('nodes', [])
            result_next = result.get('search').get('pageInfo').get('hasNextPage', False)
        
        # print(result)
        global users
        users = result_users
        print("Total GitHub Users from Cuba: %s" % len(users))

asyncio.run(main())

Total GitHub Users from Cuba: 964


In [3]:
for user in users:
    if user:
        user['followers'] = (
            user.get('followers').get('totalCount', 0)
        )
        user['following'] = (
            user.get('following').get('totalCount', 0)
        )
        user['contributions'] = (
            user.get('contributionsCollection').get('totalCommitContributions', 0)
             + user.get('contributionsCollection').get('totalPullRequestContributions', 0)
             + user.get('contributionsCollection').get('totalPullRequestReviewContributions', 0)
        )
        del(user['contributionsCollection'])

df = pd.DataFrame(users)
df.dropna(inplace=True)
# df = df.reset_index(drop=True)
df = df.sort_values(by='contributions', ascending=False)

# Top Ten Cuba
new_dtypes = {
    'followers': np.int64,
    'following': np.int64,
    'contributions': np.int64
}
position = ['\U0001F947', '\U0001F948', '\U0001F949' ] + list(range(4, TOP + 1))

df_top_ten = df[:TOP]
df_top_ten = df_top_ten.astype(new_dtypes)
# Re-Order Columns
df_top_ten = df_top_ten[
    ['name', 'login', 'location', 'contributions',
     'followers', 'following', 'bio', 'url']
]
df_top_ten.insert(0, '#', position)
print('Top %s of %s GitHub Users from Cuba' % ( TOP, len(users)))
print('Generated at: %s UTC' % datetime.utcnow())
HTML(df_top_ten.to_html(index=False))

Top 10 of 964 GitHub Users from Cuba
Generated at: 2020-08-23 00:45:26.820993 UTC


#,name,login,location,contributions,followers,following,bio,url
🥇,Maikel Llamaret Heredia,llamaret,Cuba,4190,19,18,Creator of the SWL-X Project. Founder of Web & Media Integrated Solutions. Currently co administrator of the Web of the Users Group of Free Technologies in Cuba,https://github.com/llamaret
🥈,Leynier Gutiérrez González,leynier,"La Habana, Cuba",2748,41,45,Computer Science Student and Full Stack Developer,https://github.com/leynier
🥉,Phaser Editor 2D,PhaserEditor2D,Cuba,1603,28,28,Phaser Editor 2D official account.\r\n\r\nIDE for HTML5 2D game development.,https://github.com/PhaserEditor2D
4,Guillermo,wil92,Cuba,1380,11,19,happy coding!,https://github.com/wil92
5,Luis Antonio Correa Leyva,correaleyval,"Santiago de Cuba, Cuba",1116,34,40,Computer Scientist,https://github.com/correaleyval
6,Oleksis Fraga Menéndez,oleksis,Cuba,902,1,2,,https://github.com/oleksis
7,Erich Garcia Cruz,n3omaster,Cuba,886,13,3,"PHP Developer, innovation lover, improvement lover, studying lover.",https://github.com/n3omaster
8,Asiel Díaz Benítez,adbenitez,Cuba,855,20,15,"Computer Scientist. Likes: bash, emacs, python, web, email, free/libre software, GNU project.",https://github.com/adbenitez
9,Jose Antonio Pereiro Morejon,josePereiro,"Havana, Cuba",827,0,1,,https://github.com/josePereiro
10,Marcos Antonio Maceo,stdevMac,"Havana, Cuba",774,9,19,,https://github.com/stdevMac
